# Erik's run script for the $Z_2$ gauge theory with Fermions

**Authors:**
- Clement Charles
- Erik Gustafson
- Elizabeth Hardt
- Florian Herren
- Norman Hogan
- Henry Lamm
- Sara Starecheski
- Ruth Van der Water
- Mike Wagman

The following code brings together all the necessary algorithms together to the simulations for our production run.

In [1]:
#Standard modules
import sys
import numpy as np
import yaml
import pandas
#Qiskit modules
from qiskit import IBMQ, QuantumCircuit, transpile, QuantumRegister, execute
from qiskit.providers.ibmq import RunnerResult
from qiskit import Aer
from qiskit.providers import aer
from qiskit.providers.aer import noise
from qiskit.providers.ibmq import least_busy
# from qiskit.ignis.mitigation.measurement import complete_meas_cal, CompleteMeasFitter
from qiskit.transpiler.passes.scheduling.alignments import check_durations
from qiskit.visualization import plot_histogram

import datetime
#Project modules
sys.path.append('./modules/')
sys.path.append('./inputs/')
sys.path.append('./plots/')
import StatePrep as sprep
import DD_PassManager as dd_pm
import paulitwirlingpass as pts
from Z2analysis import nsites2nqubits
import Trotterization as Trot

# for plotting
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from IPython.display import Image
plt.rcParams["figure.autolayout"] = True
colors = list(mcolors.TABLEAU_COLORS.keys())
%matplotlib inline

#For testing reload modules after edits
from importlib import reload
%load_ext autoreload

# Get input parameters from `yaml` file

In [29]:
### get run parameters from YAML input file ###

stream = open('inputs/Z2_input_jakarta_3qubits_params2.yaml', 'r')
dict_in = yaml.safe_load(stream)
stream.close()

account_info = dict_in['account_info'] # dictionary of account information
backend_opts = dict_in['backend_opts']   # dictionary of machine/simulator choices
simulation_opts = dict_in['simulation_opts'] # dictionary of lattice-simulation parameters
runtime_opts = dict_in['runtime_opts'] # dictionary of runtime parameters

### transpilation parameters ###
initial_layout = simulation_opts['layout']['initial_layout']
### Lattice parameters ###
mass = simulation_opts['lattice_params']['mass']       #fermion mass in lattice-spacing units
nsites = simulation_opts['lattice_params']['nsites']   #must be even for staggered simulation
nqubits = nsites2nqubits(nsites)
print(nqubits, nsites)
### Time evolution ###
tf = simulation_opts['time_evolution']['tf']       # total time interval in lattice-spacing units
ntrotter = simulation_opts['time_evolution']['ntrotter'] # number of trotter steps
epsilon = tf/ntrotter                              # step size in lattice-spacing units

### Pauli Twirling ###
twirl = simulation_opts['pauli_twirling']['twirl']     # if True, apply Pauli twirling
ntwirls = simulation_opts['pauli_twirling']['ntwirls'] # number of Pauli twirls

### dynamic decoupling ###
dd_method = simulation_opts['dynamic_decoupling']['method'] # if not none apply a given dynamic decoupling method

### Richardson ??? ###
richardson_level = simulation_opts['richardson_level'] # number of CNOTs for Richardson smearing

3 2


# Initialize IBM Account

In [30]:
### initialize IBM Quantum account ###

IBMQ.load_account()
hub = 'ibm-q-research'#account_info['hub']
group = 'fermilab-1'#account_info['group']
project = 'main'#account_info['project']
provider = IBMQ.get_provider(hub=hub, group=group, project=project)

ibmqfactory.load_account:WARNING:2023-01-05 08:23:52,769: Credentials are already in use. The existing account in the session will be replaced.


In [31]:
IBMQ.providers()

[<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>,
 <AccountProvider for IBMQ(hub='ibm-q-ncsu', group='nc-state', project='scattering-simul')>,
 <AccountProvider for IBMQ(hub='ibm-q-research', group='fermilab-1', project='main')>,
 <AccountProvider for IBMQ(hub='ibm-q-education', group='fermilab-1', project='qjs-for-hep')>]

# Choose target backend for simulations to run on

In [32]:
def get_backend(provider, machine, nqubits, simulator, sim_noise):
    
    ## Run on IBMQ qasm simulator
    if (simulator):
        backend = provider.get_backend('ibmq_qasm_simulator')
        
        if (sim_noise != "None"):
            device_backend = provider.get_backend(sim_noise)
            noise_model = noise.NoiseModel.from_backend(device_backend)
            backend.set_options(noise_model=noise_model.to_dict())
            
        print(f'Running on ibmq_qasm_simulator with {sim_noise} noise_model\n') 
        return backend
    ## Run on specified quantum computer(s)
    else:
#         devices = provider.backends(filters=lambda x: not (x.name() == 'ibmq_casablanca' or x.name() == 'ibmq_armonk')
#                                     and nqubits <= x.configuration().n_qubits <= 7
#                                     and not x.configuration().simulator
#                                     and x.status().operational == True)
#         print(type(devices))
#         print(type(devices[0]))
        #print(f'Backends matching critera ares {devices}\n')
        #print(device_names)
        
        if (machine == 'least_busy'):
            machine = str(least_busy(devices))
            backend = provider.get_backend(machine)    
        
        elif (machine[0:4] == 'ibmq_'):
            backend = provider.get_backend(machine)   
            
        else:
            backend = provider.get_backend(str(machine))
            
        print(f'Running on machine {machine}\n')
        return backend
        

In [33]:

# machine
# print(backend_opts['machine'])
machine = backend_opts['machine']           # machine name or "least busy"
simulator = False#backend_opts['qasm_simulator']  # if True, use ibmq_qasm_simulator 
noise_model = None#backend_opts['noise_model']   # None or device noise model 
print(noise_model)
backend = get_backend(provider, machine, nqubits, simulator, noise_model)

None
Running on machine ibmq_jakarta



# Assign Runtime Program (and inputs)

assigns values from `yaml` file

In [34]:
# number of shots per simulation
nshots = runtime_opts['nshots'] 

# Not really sure what this is needed for...
program_id = runtime_opts['program_id']

# correct for readout error using calibration matrix if True
measurement_error_mitigation = runtime_opts['readout_error_mitigation'] 

#save results for each shot if True
memory = runtime_opts['memory'] 

# Degree of layout optimization by transpiler
# 1 is least optimized; 3 is most optimized
optimization_level = runtime_opts['optimization_level'] 

In [35]:
nsites2nqubits(nsites), nsites

(3, 2)

In [36]:
def build_circuit(nq, nsteps, epsilon, mass, init_state=None,
                  dynamic_decoupling=False, Twirling=False):
    
#     print(nq)
    if init_state is None:
        if nq == 3:
            qc = QuantumCircuit(nq, nq)
            qc.h(0)
            qc.cx(0, 1)
            qc.cx(0, 2)
            qc.x(2)
            qc.h(1)
#             qc = sprep.prepare_states_nsites2(nq)['meson']
        elif nq == 7:
            qc = QuantumCircuit(nq, nq)
            qc.h(2)
            qc.cx(2, 3)
            qc.cx(3, 4)
            qc.x([2, 6])
            qc.h([1, 3, 5])
#             qc = sprep.prepare_states_nsites4(nq)['meson']

        qc.append(Trot.trotter_evolution(nsites, epsilon, mass, nsteps,
                                richardson_level=1),
                   [i for i in range(nq)])
#         for i in range(4):
        qc.h([i for i in range(0, nq, 2)])
        qc.measure([i for i in range(nq)], [i for i in range(nq)])
        return qc
#         return transpile(qc.decompose(), basis_gates=['cx', 'rz', 'sx'])
    else:
        print('not implemented')
        

# assemble circuits to run

In [37]:
machine

'ibmq_jakarta'

In [38]:
circuits_no_dd = []
circuits_w_dd = []
tvals = []
twirl = True
for i in range(1, ntrotter + 1):
    print(i)
    # with dynamic decoupling
    qc = build_circuit(nqubits, i, epsilon, mass)
    if twirl:
        qcs = pts.randomly_compile(qc, ncopy=ntwirls, backend=backend,
                                   initial_layout=initial_layout)
        qcs = transpile(qcs, basis_gates=['cx', 'rz', 'sx', 'sxdg'], optimization_level=optimization_level)
        qcs = dd_pm.DD_PassManager(qcs, DD_sequence='CPMG',
                                   providerstr=[hub, group, project],
                                   backendstr=machine)
        circuits_w_dd += qcs
    else:
        qcs = transpile(qc, basis_gates=['cx', 'rz', 'sx'], optimization_level=optimization_level,
                        initial_layout=initial_layout)
        qcs = dd_pm.DD_PassManager(qcs, DD_sequence='XY4',
                                   providerstr=[hub, group, project],
                                   backendstr=machine)
        circuits_w_dd.append(qcs)
    # add rescaling circuit
    qc = build_circuit(nqubits, i, 0, mass)
    if twirl:
        qcs = pts.randomly_compile(qc, ncopy=ntwirls, backend=backend,
                                   initial_layout=initial_layout)
        qcs = transpile(qcs, basis_gates=['cx', 'rz', 'sx', 'sxdg'], optimization_level=optimization_level)
        qcs = dd_pm.DD_PassManager(qcs, DD_sequence='CPMG',
                                   providerstr=[hub, group, project],
                                   backendstr=machine)
        circuits_w_dd += qcs
    else:
        qcs = transpile(qc, basis_gates=['cx', 'rz', 'sx'], optimization_level=optimization_level,
                        initial_layout=initial_layout)
        qcs = dd_pm.DD_PassManager(qcs, DD_sequence='XY4',
                                   providerstr=[hub, group, project],
                                   backendstr=machine)
        circuits_w_dd.append(qcs)
    
circuits = circuits_w_dd

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


In [39]:
circuits_readout = []
qc = QuantumCircuit(nqubits, nqubits)
qc.measure_all()
circuits_readout.append(qc)
qc = QuantumCircuit(nqubits, nqubits)
qc.x([i for i in range(nqubits)])
qc.measure_all()
circuits_readout.append(qc)
circuits_readout = transpile(circuits_readout, initial_layout=initial_layout)

In [40]:
circuits = circuits + circuits_readout

In [41]:
len(circuits)

1202

In [27]:
job1 = backend.run(circuits, job_name='dynamic decoupling no twirling', shots=nshots)

In [28]:
jobs = [job1]
dataframe  = pandas.DataFrame(columns=['jobid', 'machine', 'date', 'sim type', 'nt', 'pauli twirling', 'dynamic decoupling', 'epsilon', 'mass', 'counts bare'])
data_row = []
ntwirls = 1
for k in range(len(jobs)):
    for j in range(42):
        data_dict = {}
        data_dict['jobid'] = jobs[k].job_id()
        data_dict['machine'] = jobs[k].backend().name()
        data_dict['date'] = datetime.datetime.today()
        if (j // ntwirls) % 2 == 0:
            data_dict['sim type'] = 'evolution'
            data_dict['nt'] = j // (ntwirls * 2) + 1
            data_dict['pauli twirling'] = twirl
            data_dict['dynamic decoupling'] = True
            data_dict['epsilon'] = epsilon
            data_dict['mass'] = mass
            data_dict['counts bare'] = jobs[k].result().results[j].data.counts
#             data_dict['counts mitigated'] = get_quasi_probabilities(jobs[k].result().results[i].data.counts)
            data_row.append(data_dict)
        else:
            data_dict['sim type'] = 'rescaling'
            data_dict['nt'] = j // (ntwirls * 2) + 1
            data_dict['pauli twirling'] = twirl
            data_dict['dynamic decoupling'] = True
            data_dict['epsilon'] = epsilon
            data_dict['mass'] = mass
            data_dict['counts bare'] = jobs[k].result().results[j].data.counts
#             data_dict['counts mitigated'] = get_quasi_probabilities(jobs[k].result().results[i].data.counts)
            data_row.append(data_dict)
        
dataframe = pandas.DataFrame(data_row)

dataframe.to_csv(f'no_twirl_dd_simulation_production_run_on_{machine}_data={datetime.datetime.today().date()}_{nqubits}qubits_{jobs[0].job_id()}.csv')

# build the circuit jobs

In [42]:
job1 = backend.run(circuits[:300], job_name='dynamic decoupling steps 1 - 5', shots=nshots)

In [43]:
job2 = backend.run(circuits[300:600], job_name='dynamic decoupling steps 6 - 10', shots=nshots)

In [44]:
job3 = backend.run(circuits[600:900], job_name='dynamic decoupling steps 11 - 15', shots=nshots)

In [47]:
job4 = backend.run(circuits[900:1200], job_name='dynamic decoupling steps 16 - 20', shots=nshots)

In [46]:
job5 = backend.run(circuits_readout, job_name='readout correction circuits', shots=nshots)

In [15]:
job1.status()

<JobStatus.QUEUED: 'job is queued'>

In [12]:

import qiskit
from qiskit import QuantumRegister,ClassicalRegister,QuantumCircuit,execute
from qiskit.compiler import transpile
import numpy as np
from numpy import pi as pi
import random
from qiskit import IBMQ
from qiskit.circuit.library import XGate, RZGate
from qiskit.transpiler import PassManager, InstructionDurations
from qiskit.transpiler.passes import ALAPSchedule, DynamicalDecoupling

In [ ]:
calibration_matrix = np.array([1])
for j in range(nqubits):
    matrix = np.zeros((2, 2))
    for key in job5.result().get_counts(0):
        if key[j] == '0':
            matrix[0, 0] += job5.result().get_counts(0)[key]
        else:
            matrix[1, 0] += job5.result().get_counts(0)[key]
    for key in job5.result().get_counts(1):
        if key[j] == '0':
            matrix[0, 1] += job5.result().get_counts(1)[key]
        else:
            matrix[1, 1] += job5.result().get_counts(1)[key]
    matrix /= nshots
    matrix = np.linalg.inv(matrix)
    calibration_matrix = np.kron(matrix, calibration_matrix)
    

In [1]:
def get_quasi_probabilities(counts):
    quasi_probs = np.zeros(2 ** nqubits)
    raw_counts = np.zeros(2 ** nqubits)
    for key in counts.keys():
        raw_counts[int(key, 16)] += counts[key] / nshots
    quasi_probs = calibration_matrix @ raw_counts
    return quasi_probs / np.sum(quasi_probs)

# build the runtime-object

In [49]:
jobs = [job1, job2, job3, job4, job5]

In [50]:
dataframe  = pandas.DataFrame(columns=['jobid', 'machine', 'date', 'sim type', 'nt', 'pauli twirling', 'dynamic decoupling', 'epsilon', 'mass', 'counts bare'])
data_row = []
for k in range(len(jobs)):
    if k == 4:
        for j in range(2):
            data_dict = {}
            data_dict['jobid'] = jobs[k].job_id()
            data_dict['machine'] = jobs[k].backend().name()
            data_dict['date'] = datetime.datetime.today()
            if (j // ntwirls) % 2 == 0:
                data_dict['sim type'] = 'evolution'
                data_dict['nt'] = j // (ntwirls * 2) + 1
                data_dict['pauli twirling'] = twirl
                data_dict['dynamic decoupling'] = True
                data_dict['epsilon'] = epsilon
                data_dict['mass'] = mass
                data_dict['counts bare'] = jobs[k].result().results[j].data.counts
    #             data_dict['counts mitigated'] = get_quasi_probabilities(jobs[k].result().results[i].data.counts)
                data_row.append(data_dict)
            else:
                data_dict['sim type'] = 'rescaling'
                data_dict['nt'] = j // (ntwirls * 2) + 5 * k + 1
                data_dict['pauli twirling'] = twirl
                data_dict['dynamic decoupling'] = True
                data_dict['epsilon'] = epsilon
                data_dict['mass'] = mass
                data_dict['counts bare'] = jobs[k].result().results[j].data.counts
    #             data_dict['counts mitigated'] = get_quasi_probabilities(jobs[k].result().results[i].data.counts)
                data_row.append(data_dict)
    else:
        for j in range(300):
            data_dict = {}
            data_dict['jobid'] = jobs[k].job_id()
            data_dict['machine'] = jobs[k].backend().name()
            data_dict['date'] = datetime.datetime.today()
            if (j // ntwirls) % 2 == 0:
                data_dict['sim type'] = 'evolution'
                data_dict['nt'] = j // (ntwirls * 2) + 1
                data_dict['pauli twirling'] = twirl
                data_dict['dynamic decoupling'] = True
                data_dict['epsilon'] = epsilon
                data_dict['mass'] = mass
                data_dict['counts bare'] = jobs[k].result().results[j].data.counts
    #             data_dict['counts mitigated'] = get_quasi_probabilities(jobs[k].result().results[i].data.counts)
                data_row.append(data_dict)
            else:
                data_dict['sim type'] = 'rescaling'
                data_dict['nt'] = j // (ntwirls * 2) + 5 * k + 1
                data_dict['pauli twirling'] = twirl
                data_dict['dynamic decoupling'] = True
                data_dict['epsilon'] = epsilon
                data_dict['mass'] = mass
                data_dict['counts bare'] = jobs[k].result().results[j].data.counts
    #             data_dict['counts mitigated'] = get_quasi_probabilities(jobs[k].result().results[i].data.counts)
                data_row.append(data_dict)
        
dataframe = pandas.DataFrame(data_row)

In [ ]:
dataframe.to_csv(f'simulation_production_run_on_{machine}_data={datetime.datetime.today().date()}_{nqubits}qubits_{jobs[0].job_id()}.csv')